# Importing libraries

In [1]:
import requests
import csv
import time
from bs4 import BeautifulSoup

In [2]:
# sub-reddit url and header to imitate browser request
url = "https://old.reddit.com/r/india/"
headers = {'User-Agent': 'Microsoft Edge 41.16299.1004.0'}

#requesting page
page = requests.get(url, headers=headers)

In [3]:
#making an object of BeautifulSoup as html parser
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
#scraping page to find web domain of all headings
domains = soup.find_all("span",class_ = "domain")

In [5]:
#printing all domains and also finding their parent using DOM structure of page
for domain in domains:
    parent_div = domain.parent.parent.parent.parent
    print(parent_div.text)
    print(domain.text)

166167168CoronavirusCoronavirus (COVID-19) Megathread - News and Updates - 4 (self.india)submitted 7 days ago * by IAmMohitBigotry begets Bigotry[M] - announcement2613 commentssharesavehidereportloading...
(self.india)
818283Scheduled[Monthly Happiness Thread] Randians, please share a good/positive/happy news happening in your life recently ! :) (self.india)submitted 3 days ago by cool_boyy - announcement105 commentssharesavehidereportloading...
(self.india)
1179517961797PoliticsMumbai Police Meri Jaan (i.redd.it)submitted 5 hours ago by Fabulous-RiceRealized my Lifestyle is called Social Distancing30 commentssharesavehidereportloading...
(i.redd.it)
2292529262927Non-PoliticalAn aerial view of Gangaikonda Cholapuram Temple (i.redd.it)submitted 17 hours ago by jatadharius76 commentssharesavehidereportloading...
(i.redd.it)
3221222223PoliticsArnab Goswami alleges physical assault by Congress, calls Sonia Gandhi a ‘Coward’ (theweek.in)submitted 5 hours ago by Dhinnu101 commentssharesavehi

In [6]:
#Finding title,author,likes,flair,data domain of one page
attrs = {'class' : 'thing'}

for post in soup.find_all('div',attrs = attrs):
    title = post.find('p', class_="title").a.text
    print(title)
    author = post.find('a', class_='author').text
    likes = post.find("div", class_ = "score likes").text
    if likes == "•":
        likes = "None"
    flair = post.find("span",class_ = "linkflairlabel")
    if flair !=None:
        flair = flair.text

Coronavirus (COVID-19) Megathread - News and Updates - 4
[Monthly Happiness Thread] Randians, please share a good/positive/happy news happening in your life recently ! :)
Mumbai Police Meri Jaan
An aerial view of Gangaikonda Cholapuram Temple
Arnab Goswami alleges physical assault by Congress, calls Sonia Gandhi a ‘Coward’
Thane Man Allegedly Refuses To Take Delivery From Muslim Man, Arrested
'Treat Sanitation Workers Like Health Workers, Pay Them At Least Rs 20,000 Per Month'
Give Us Kerala Model Over Gujarat Model, Any Day - By Ramachandra Guha
"BJP Continues To Spread Virus Of Communal Prejudice": Sonia Gandhi
Govt should revise list of "reserved" categories to keep well-off out: SC
Pakistani Twitter account posing as Omani princess shares anti-India material, old tweets show ISPR's role
Kerala govt has decided to deduct the salary of state employees for 6 days in a month for next 5 months and of MLAs and other legislators by 30% for next one year to tide over the unprecedented fina

In [12]:
#Some values for writing in csv file
counter = 0                             ## to give unique id to each row
count = 1
title_check = []                        ## to check there are no repeated titles in data
attrs = {'class' : 'thing'}             ## attribute to check for the heading

In [29]:
## Finding title,author,likes,flair,data domain of one page and saving it in csv

while(counter<=500):
    posts = soup.find_all('div',attrs = attrs)
    for post in posts:
        title = post.find('p', class_="title").a.text
        if title not in title_check:
            title_check.append(title)
        else: 
            continue
        
        try:
            author = post.find("a", class_="author").text
        except:
            author = "[deleted]"
        
        likes = post.find("div", class_ = "score likes").text
        if likes == "•":
            likes = 0
        if type(likes)!=int and likes[-1]=="k":
            likes = float(likes[:-1])*1000
        likes = int(likes)
        
        flair = post.find("span",class_ = "linkflairlabel")
        if flair!=None:
            flair = flair.text
        else:
            continue
        
        data_domain = post.attrs['data-domain']
        post_line = [counter, title, author, likes, data_domain,flair]
        
        with open('out.csv', 'a',encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(post_line)

        counter += 1
        
    ##Finding next-button in page
    next_button = soup.find("span", class_="next-button")
    # if not found, jump to another page
    if next_button != None:
        next_page_link = next_button.find("a").attrs['href']
    elif count==1 : 
        next_page_link = "https://old.reddit.com/r/india/controversial/?sort=controversial&t=year"
        
    #Important : Not to use all resources of a website by requesting a large number of pages in small amount of time
    #Following guidelines of web scraping a website
    #time.sleep(2)
    
    page = requests.get(next_page_link, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')